#Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset Clean up

In [ ]:
pip install Pillow

In [ ]:
!ls /content/drive/MyDrive/Colab_Inputs/Fantasy_Landscapes/ | wc -l

794


Take the number and add 1. Now place it beneath in range (1, ___).
Convert to jpg:

In [ ]:
from PIL import Image
!mkdir jpgs
for a in range(1, 253): #adjust the number to count through all the images
  im1 = Image.open('/content/drive/MyDrive/Colab_Inputs/Fantasy_Landscapes/_Pngs/Image' + str(a) + '.png')
  rgb_im = im1.convert('RGB') #if there is an alpha canal bc jpg has no transparency
  rgb_im.save('/content/drive/MyDrive/Colab_Inputs/Fantasy_Landscapes/Image' + str(a) + '.jpg')

mkdir: cannot create directory ‘jpgs’: File exists


Now from Derricks Notebook.
Install Repo for dataset-tools:

In [ ]:
!git clone https://github.com/dvschultz/dataset-tools

Cloning into 'dataset-tools'...
remote: Enumerating objects: 416, done.
remote: Counting objects: 100% (229/229), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 416 (delta 124), reused 140 (delta 58), pack-reused 187
Receiving objects: 100% (416/416), 687.02 KiB | 8.38 MiB/s, done.
Resolving deltas: 100% (231/231), done.


## Let’s Normalize our Data!

In [ ]:
#count how many files are in our folder to start with
!ls /content/drive/MyDrive/Colab_Inputs/Fantasy_Landscapes/ | wc -l

794


### Remove anything too small

In [ ]:
!python /content/dataset-tools/sort.py -h

In [ ]:
!python ./dataset-tools/sort.py \
-i /content/drive/MyDrive/Colab_Inputs/Fantasy_Landscapes/ \
-o /content/sorted/ \
--min_size 256 \
--max_size 100000

Processing folder: /content/drive/MyDrive/Colab_Inputs/Fantasy_Landscapes/


In [ ]:
!ls /content/sorted/exclude_256-100000 | wc -l

624


### Remove ~Duplicates
Note: This process is quite slow.

In [ ]:
!python ./dataset-tools/dedupe.py -h

This command will only remove exact duplicates:

In [ ]:
!python ./dataset-tools/dedupe.py \
--input_folder /content/sorted/exclude_256-100000 \
--output_folder /content/deduped/

In [ ]:
!ls /content/deduped/exclude | wc -l

616


### Crop/Pad to Square

This is the most consequential step. How you choose to crop your images to square will have a great effect on your output so I recommed thinking about how to approach this (if you have the time/energy to do it manually that will almost always be your best choice).

In [ ]:
!python ./dataset-tools/dataset-tools.py -h

In [ ]:
!python -p -h

Unknown option: -p
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.


This most important command here is the `--process_type` command. Some important processes here are:

- `resize`: resize to a common size (no cropping)
- `crop_to_square`: crop every image to a square
- `square`: pad (add borders) to images
- `crop`: crop image to a non-square dimension

This command will crop your image to square from the center. It’s the most commmon techninque.

In [ ]:
!python ./dataset-tools/dataset-tools.py \
-i /content/deduped/exclude \
-o /content/cropped/ \
-p crop_to_square \
--max_size 1024 \
--file_extension jpg
#for testing #-o /content/drive/MyDrive/2dtodepth/infile \#683,512

Processing folder: /content/deduped/exclude


Move the pictures to infile folder for next step:

In [ ]:
! mv /content/cropped/sq-1024/*.jpg /content/drive/MyDrive/2dtodepth/infile/

rename the files

In [ ]:
data = "/content/drive/MyDrive/2dtodepth/infile/"
import numpy as np
import os
ldseg=np.array(os.listdir(data))

#Function to rename multiple files
def main():
  i = 1
  for filename in ldseg:
    dst ="Image_"+str(i)+ ".jpg"
    src =data+ filename
    dst =data+ dst
    os.rename(src, dst)
    i += 1
if __name__ == '__main__':
  main()

Check if a weird folder appeared!!

In [ ]:
!ls /content/drive/MyDrive/2dtodepth/infile/ | wc -l

616


#Depth map


create depth map:

In [ ]:
!gdown --id 1HLOp55ntZjtpG3wet37fynOE4PNQSqMj -O /content/2dtodepth.zip

In [ ]:
!unzip /content/2dtodepth.zip -d /content/

In [ ]:
%cd /content/2dtodepth
!python 2dtodepth.py --input=single_view

/content/drive/MyDrive/2dtodepth
------------ Options -------------
batchSize: 8
beta1: 0.5
checkpoints_dir: ./checkpoints/
continue_train: False
display_freq: 100
display_id: 1
display_winsize: 256
fineSize: 256
gpu_ids: [0, 1, 2, 3]
human_data_term: 0
identity: 0.0
input: single_view
isTrain: True
lambda_A: 10.0
lambda_B: 10.0
loadSize: 286
lr: 0.0004
lr_decay_epoch: 8
lr_policy: step
max_dataset_size: inf
mode: Ours_Bilinear
model: pix2pix
nThreads: 2
name: test_local
ndf: 64
ngf: 64
niter: 100
niter_decay: 100
no_flip: False
no_html: False
no_lsgan: False
norm: instance
output_nc: 3
phase: train
pool_size: 50
print_freq: 100
save_epoch_freq: 5
save_latest_freq: 5000
serial_batches: False
simple_keypoints: 0
use_dropout: False
which_epoch: latest
which_model_netG: unet_256
-------------- End ----------------
========================= Video dataset #images = 616 =========
======================================  DIW NETWORK TRAIN FROM Ours_Bilinear=======================
=============

In [ ]:
!ls /content/drive/MyDrive/2dtodepth/outfile/ | wc -l

616



Take the number and add 1. Now place it beneath in range (1, ___).

Crop original image away:

In [ ]:
# Importing Image class from PIL module
from PIL import Image
for b in range(1, 617):
  # Opens a image in RGB mode
  im = Image.open('/content/drive/MyDrive/2dtodepth/outfile/Image_' + str(b) + '.jpg')
 
  # Size of the image in pixels (size of original image)
  # (This is not mandatory)
  width, height = im.size
 
  # Setting the points for cropped image
  left = width/2
  top = 0
  right = width
  bottom = height
 
  # Cropped image of above dimension
  # (It will not change original image)
  im1 = im.crop((left, top, right, bottom))
  !mkdir /content/drive/MyDrive/2dtodepth/depthmap/
  im1.save('/content/drive/MyDrive/2dtodepth/depthmap/Image_' + str(b) + '.jpg')

mkdir: cannot create directory ‘/content/drive/MyDrive/2dtodepth/depthmap/’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/2dtodepth/depthmap/’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/2dtodepth/depthmap/’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/2dtodepth/depthmap/’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/2dtodepth/depthmap/’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/2dtodepth/depthmap/’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/2dtodepth/depthmap/’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/2dtodepth/depthmap/’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/2dtodepth/depthmap/’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/2dtodepth/depthmap/’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/2dtodepth/depthmap/’: File exists
mkdir: cannot create 

#Create folder for training

In [ ]:
!mkdir /content/drive/MyDrive/Colab_Inputs/Fantasy_Landscapes/Training_Dataset/
!mkdir /content/drive/MyDrive/Colab_Inputs/Fantasy_Landscapes/Training_Dataset/train

In [ ]:
!cp /content/drive/MyDrive/2dtodepth/outfile/*.jpg /content/drive/MyDrive/Colab_Inputs/Fantasy_Landscapes/Training_Dataset/train

In [ ]:
#%cp -av /content/drive/MyDrive/2dtodepth/depthmap/*.jpg /content/drive/MyDrive/Colab_Inputs/Fantasy_Landscapes/Training_Dataset

In [ ]:
#%cp -av /content/drive/MyDrive/2dtodepth/infile/*.jpg /content/drive/MyDrive/Colab_Inputs/Fantasy_Landscapes/Training_Dataset